# Tesla Stock Prediction Preprocessing

Preprocessing has moved to this separate file for added efficiency. Preprocessing is done and then the resulting dataframe is saved to a file so that it doesn't need to be repeated unless changes are made to the preprocessing process.

I use Trump and Elon Musk tweet data as part of the prediction. I did some low effort sentiment analysis using the nlkt Python library (nltk = Natural Language Toolkit) and the VADER pre-trained sentiment analysis tool. I also make my own attempt at analysis of the tweets by seeing whether they contain some Tesla-related keywords. In addition I use Google Trends data for the keyword "Tesla" as another feature. The result of this preprocessing is saved as "preprocessed.csv"

In [12]:
import nltk
import pandas as pd
import numpy as np

In [13]:
from nltk.sentiment import SentimentIntensityAnalyzer
from statistics import mean
sia = SentimentIntensityAnalyzer()

# These are the functions for the sentiment analysis. 
# is_positive uses the pre-made SentimentIntensityAnalyzer that's a part of the nltk library.
def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    scores = [sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(tweet)]
    return mean(scores) > 0

# contains_tesla returns positive if the tweet contains any of the words or phrases in the "search" list.
def contains_tesla(tweet: str) -> bool:
    search = ("tesla", "tsla", "cybertruck", "model s", "model x", "model 3", "model y", "electric car", "electric cars")
    return any(word in tweet.lower() for word in search)

In [14]:
df_trump = pd.read_csv("Data/trump_tweets.csv", parse_dates = ['date'], index_col=7)
# Applying the sentiment analysis functions to the trump data
df_trump["Trump Positive Sentiment"] = df_trump.apply(lambda row: is_positive(row["text"]), axis = 1)
df_trump["Trump Contains Tesla"] = df_trump.apply(lambda row: contains_tesla(row["text"]), axis=1)
print(df_trump.shape)
df_trump.head()

(56571, 10)


,id,text,isRetweet,isDeleted,device,favorites,retweets,isFlagged,Trump Positive Sentiment,Trump Contains Tesla
date,,,,,,,,,,
2009-05-04 18:54:00,1.698309e+09,Be sure to tune in and watch Donald Trump on L...,f,f,Twitter Web Client,939,519,f,True,False
2009-05-05 01:00:00,1.701461e+09,Donald Trump will be appearing on The View tom...,f,f,Twitter Web Client,259,34,f,True,False
2009-05-08 13:38:00,1.737480e+09,Donald Trump reads Top Ten Financial Tips on L...,f,f,Twitter Web Client,37,15,f,True,False
2009-05-08 20:40:00,1.741161e+09,New Blog Post: Celebrity Apprentice Finale and...,f,f,Twitter Web Client,29,11,f,False,False
2009-05-12 14:07:00,1.773561e+09,"""""""My persona will never be that of a wallflow...",f,f,Twitter Web Client,1877,1321,f,False,False


In [15]:
df_musk = pd.read_csv("Data/musk_2021.csv", parse_dates = ['date'], index_col=0)
# The csv is indexed by id, we want it indexed by date
df_musk.set_index("date", inplace=True)
df_musk.sort_index(ascending=True, inplace=True)
df_musk["Musk Positive Sentiment"] = df_musk.apply(lambda row: is_positive(row["tweet"]), axis = 1)
df_musk["Musk Contains Tesla"] = df_musk.apply(lambda row: contains_tesla(row["tweet"]), axis=1)
print(df_musk.shape)
df_musk.head()

(12339, 39)


,id,conversation_id,created_at,timezone,place,tweet,language,hashtags,cashtags,user_id,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Musk Positive Sentiment,Musk Contains Tesla
date,,,,,,,,,,,,,,,,,,,,,
2010-06-04 18:31:57,15434727182,15434727182,1.275676e+12,0,NaN,"Please ignore prior tweets, as that was someon...",en,[],[],44196397,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,True,False
2011-12-01 09:55:11,142179928203460608,142179928203460608,1.322733e+12,0,NaN,Went to Iceland on Sat to ride bumper cars on ...,en,[],[],44196397,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,True,False
2011-12-01 10:29:04,142188458125963264,142188458125963264,1.322735e+12,0,NaN,I made the volume on the Model S http://t.co/...,en,[],[],44196397,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,False,True
2011-12-03 08:20:28,142880871391838208,142880871391838208,1.322900e+12,0,NaN,"Great Voltaire quote, arguably better than Twa...",en,[],[],44196397,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,True,False
2011-12-03 08:22:07,142881284019060736,142881284019060736,1.322901e+12,0,NaN,That was a total non sequitur btw,en,[],[],44196397,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,False,False


In [16]:
# Tesla stock data from Yahoo Finance
df_tsla = pd.read_csv("Data/TSLA.csv", parse_dates=['Date'], index_col=0)
print(df_tsla.shape)
df_tsla.head()

(2856, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-30,5.158,6.084,4.660,4.766,4.766,85935500
2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000
2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000
2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500
2010-07-07,3.280,3.326,2.996,3.160,3.160,34608500


In [17]:
# Google Trends Data for the keyword "Tesla"
# Explanation of columns copied from dataDownload.py:
"""
    Contains 4 columns.
            The column named after the word argument contains the daily search
            volume already scaled and comparable through time.
            The column f'{word}_unscaled' is the original daily data fetched
            month by month, and it is not comparable across different months
            (but is comparable within a month).
            The column f'{word}_monthly' contains the original monthly data
            fetched at once. The values in this column have been backfilled
            so that there are no NaN present.
            The column 'scale' contains the scale used to obtain the scaled
            daily data.
"""
# We only need the "Tesla" column, which is scaled across the various months. Renamed "Tesla Trend" for clarity

df_tslaTrend = pd.read_csv("Data/tslaTrend.csv", parse_dates=['date'], index_col=0)
print(df_tslaTrend.shape)
df_tslaTrend.drop(columns=['Tesla_unscaled', 'Tesla_monthly', 'isPartial', 'scale'], axis=1, inplace=True)
df_tslaTrend.rename(columns={'Tesla': 'Tesla Trend'}, inplace=True)
df_tslaTrend.head()

(3928, 5)


,Tesla Trend
date,
2010-06-30,NaN
2010-07-01,7.00
2010-07-02,6.09
2010-07-03,1.19
2010-07-04,2.94


## Preparing Data

### Trimming and merging dataframes

In [18]:
# Aligning Tweet Data with Doge Data
df_musk = df_musk.loc[(df_musk.index >= '2010-06-30')]
# Dropping columns that are not expected to be useful
df_musk = df_musk.drop(columns=[
    'id', 'conversation_id', 'timezone', 'place',
    'language', 'user_id', 'geo', 'source', 'user_rt_id',
    'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
    'translate', 'trans_src', 'trans_dest', 'user_id_str',
    'username', 'name', 'day', 'hour', 'link', 'urls',
    'photos', 'video', 'thumbnail', 'retweet', 'quote_url',
    'search', 'near', 'created_at', 'tweet', 'hashtags',
    'cashtags'
], axis=1)

# Changing by-the-second data to by-the-day data
changed1 = df_musk.groupby([df_musk.index.date]).size().reset_index(name='Musk Tweet Count')
changed2 = df_musk.groupby([df_musk.index.date]).agg('sum')
changed1 = changed1.set_index('index')

df_musk = changed1.merge(changed2, how='outer', left_index=True, right_index=True)
df_musk = df_musk.rename_axis('Date')
df_musk.rename(columns={"nlikes": "Musk Likes", "nreplies": "Musk Replies", "nretweets": "Musk Retweets"}, inplace=True)
df_musk["Musk Negative Sentiment"] = df_musk["Musk Tweet Count"] - df_musk["Musk Positive Sentiment"]
print(df_musk.shape)
df_musk.head(10)

(2004, 7)


,Musk Tweet Count,Musk Likes,Musk Replies,Musk Retweets,Musk Positive Sentiment,Musk Contains Tesla,Musk Negative Sentiment
Date,,,,,,,
2011-12-01,2,267,63,24,1,1,1
2011-12-03,2,82,38,31,1,0,1
2011-12-04,1,65,17,9,1,0,0
2011-12-21,7,1330,87,597,5,0,2
2011-12-22,3,1349,132,206,1,1,2
2011-12-24,3,117113,1370,8434,2,0,1
2011-12-26,6,558,102,171,3,0,3
2011-12-27,1,39,13,34,1,0,0
2011-12-28,1,12,7,1,1,0,0


In [19]:
# Aligning Tweet Data with Doge Data
#df_trump = df_trump.loc[(df_trump.index >= '2011-12-01')]
# Dropping ID column since it's not expected to be useful
df_trump = df_trump.drop(columns=['id'], axis=1)

# Changing by-the-second data to by-the-day data
changed1 = df_trump.groupby([df_trump.index.date]).size().reset_index(name='Trump Tweet Count')
changed2 = df_trump.groupby([df_trump.index.date]).agg('sum')
changed1 = changed1.set_index('index')

df_trump = changed1.merge(changed2, how='outer', left_index=True, right_index=True)
df_trump = df_trump.rename_axis('Date')
df_trump.rename(columns={"favorites": "Trump Favorites", "retweets": "Trump Retweets"}, inplace=True)
df_trump["Trump Negative Sentiment"] = df_trump["Trump Tweet Count"] - df_trump["Trump Positive Sentiment"]

print(df_trump.shape)
df_trump.head()

(3500, 6)


,Trump Tweet Count,Trump Favorites,Trump Retweets,Trump Positive Sentiment,Trump Contains Tesla,Trump Negative Sentiment
Date,,,,,,
2009-05-04,1,939,519,1,0,0
2009-05-05,1,259,34,1,0,0
2009-05-08,2,66,26,1,0,1
2009-05-12,2,1902,1347,1,0,1
2009-05-13,1,17,16,1,0,0


In [20]:
# Aligning tsla Data with Tweet Data
df_tsla = df_tsla.loc[(df_tsla.index <= '2021-03-22')]
#df_tsla = df_tsla.loc[(df_tsla.index >= '2011-12-01')]
print(df_tsla.shape)

(2700, 6)


In [21]:
# Merging Two Data Sets Together
df = df_tsla.merge(df_musk, how='outer', left_index=True, right_index=True)
df = df.merge(df_trump, how='outer', left_index=True, right_index=True)
df = df.merge(df_tslaTrend, how='outer', left_index=True, right_index=True)
# As part of the merge we get some NA values, We want to replace those with a value of '0'
df[['Musk Tweet Count', 'Musk Likes', 'Musk Replies', 'Musk Retweets', 'Musk Positive Sentiment', \
        'Musk Contains Tesla', 'Musk Negative Sentiment', 'Trump Tweet Count', 'Trump Favorites', 'Trump Retweets', \
        'Trump Positive Sentiment', 'Trump Contains Tesla', 'Trump Negative Sentiment']] \
    = df[['Musk Tweet Count', 'Musk Likes', 'Musk Replies', 'Musk Retweets', 'Musk Positive Sentiment', \
            'Musk Contains Tesla', 'Musk Negative Sentiment', 'Trump Tweet Count', 'Trump Favorites', 'Trump Retweets', \
            'Trump Positive Sentiment', 'Trump Contains Tesla', 'Trump Negative Sentiment']].fillna(0)

# Rows with NaN values for Open, High, Low, etc will be removed later by the series_to_supervised function
df.index.name='Date'
df.dropna(inplace=True)
df.head()

,Open,High,Low,Close,Adj Close,Volume,Musk Tweet Count,Musk Likes,Musk Replies,Musk Retweets,Musk Positive Sentiment,Musk Contains Tesla,Musk Negative Sentiment,Trump Tweet Count,Trump Favorites,Trump Retweets,Trump Positive Sentiment,Trump Contains Tesla,Trump Negative Sentiment,Tesla Trend
Date,,,,,,,,,,,,,,,,,,,,
2010-07-01,5.000,5.184,4.054,4.392,4.392,41094000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16.0,27.0,0.0,0.0,1.0,7.00
2010-07-02,4.600,4.620,3.742,3.840,3.840,25699000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,36.0,78.0,2.0,0.0,0.0,6.09
2010-07-06,4.000,4.000,3.166,3.222,3.222,34334500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,47.0,0.0,0.0,1.0,2.10
2010-07-07,3.280,3.326,2.996,3.160,3.160,34608500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.78
2010-07-08,3.228,3.504,3.114,3.492,3.492,38557000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.60


In [22]:
df.to_csv('Data/preprocessed.csv')